# Mixture optimisation using machine learning

In this tutorial we will be investigating how to apply machine learning techniques to a mixture optimisation problem.

This is a important step in a 'designed experiment', where the exact values of a mixture is determined using a strategic methodology. Such mixture design experiments are often used in the fields of pharmaceutical and material design.

We will be following a study by Yeh (Journal of Materials in Civil Engineering, 18, 597–604), which is explored by Kuhn and Johnson in their book, Applied Predictive Modelling.

Concrete is an integral part of most industrialized societies. It is used to some extent in nearly all structures and in many roads. One of the main properties of interest (beside cost) is the compressive strength of the hardened concrete.

We will be using a data set which provides us with several hundred concrete mixtures and their compressive strength. Our overall aim is to build a model which can accurately predict compressive strength based on the ingredients, and then to use this model to predict the strongest mixtures, which can lead further experiment.

We will be making use of several scientific Python libraries to follow through elements of this experiment. Specifically, will be using Pandas to hold and manipulate structured data, Matplotlib to visualise our data, Scikit-Learn to build machine learning models, and Scipy to perform an optimisation.


Our objectives are:

1) Introduce Pandas, and to use the library to hold our data in a structured format

2) Use Matplotlib to visualise our data, and gain some intuition of what ingredients and relationships may be important for our model

3) Introduce Scikit-Learn, and use it to build some regression models so that we can accurately predict compressive strenght from the mixture

4) Perform an optimisation process using SciPy to create some predictions of the optimal concrete mixture

# Part 1: introducing Pandas

Those of you who have never used jupyter notebooks before, we have an interactive python session with inline documentation using markdown.

We will start by loading in our raw data using a library called pandas.

Pandas (short for panel data) gives us access to a dataframe, which is an abstraction allowing us to hold data in a more convenient object than a list or array. The dataframe is also used in languages like R and frameworks like Spark to make processing data easy. Think of it like a spreadsheet on steroids, with a huge amount of built in functionality for manipulating our data. 

Pandas is an open-sourced library. There is a huge community of data scientists and analysts who use it and contribute to it, meaning it is high quality, under continuous development, and any bugs are fixed very rapidly.

In [ ]:
# we load a library into our namespace like so
import pandas as pd

In [ ]:
# pandas can easily read in data from a number of sources
url = "https://raw.githubusercontent.com/philipmgoddard/gradnet_concrete/master/raw/concrete.csv"
concrete_raw = pd.read_csv(url, index_col = 0)

In [ ]:
# we can also get a summary of our data with the info() method
concrete_raw.info()

pandas allows us to answer practically any question we might have about the data. The API docs can be found [here]( http://pandas.pydata.org/pandas-docs/stable/api.html).

The library is huge, so unless it is all you use every day it is hard to memorise. From experience, you can begin to  understand what type of manipulations and summaries are possible, and then you can easily consult the API docs or Stack Overflow to solve your problems.

## Lets have a little play with pandas


In [ ]:
# how to we select data?

# all rows, one column
# note a 'series' is returned here
concrete_raw.loc[:, 'CompressiveStrength'].head()

In [ ]:
# select the first three rows
concrete_raw.loc[0:3, 'CompressiveStrength'].head()

In [ ]:
# pass in a list to access multiple columns (note a dataframe is returned)
concrete_raw.loc[0:3, ['Water', 'CompressiveStrength']].head()

In [ ]:
# we can apply summary functions to columns
concrete_raw.loc[:, 'CompressiveStrength'].mean()

In [ ]:
# we can peform groupings and aggregation
# what is the average compressive strength for different ages of concrete?
(
    concrete_raw
        .groupby('Age')['CompressiveStrength']
        .agg(['mean', 'count'])
        .reset_index()
        .sort_values(by='Age')   
).head()

In [ ]:
# we can query by filtering rows based on conditions
concrete_raw.loc[concrete_raw['Age'] == 28, :].head()

In [ ]:
# or a different syntax
concrete_raw.query('Age == 28').head()

In [ ]:
# we can create new columns
concrete_raw.assign(cStrengthSq = lambda x: x.CompressiveStrength ** 2).head()

## Pandas is really useful!

It has a huge set of functionality. Unfortunately it can be a bit overwhelming at first, and requires a bit of a learning curve to get used to it, and there are often multiple ways to accomplish the same task.

If you deal with a lot of data, it is far more flexible and resilient than Excel or other spreadsheet programs.

Practice is by far the best way to get to grips with it, so why not try it for some of your work? You can write scripts which run end to end to take your data from raw to processed, which means you have fully reproducible work!

## Lets do some useful work on our data now

### 1) Is there any missing data? 

We determine which rows are null using [pd.isnull](http://pandas.pydata.org/pandas-docs/version/0.22/generated/pandas.isnull.html)

We  can get a series of true or false for each row, depending if they contain nulls, like this: concrete_raw.isnull().any(axis=1)

In [ ]:
# lets use this result to filter the rows in our dataframe that contain missing data
concrete_raw.loc[concrete_raw.isnull().any(axis = 1), :]

### 2) Are there any duplicates inputs? How might we deal with these?

In [ ]:
# first, lets create a list of the feature names to easily select them

outcome_name = 'CompressiveStrength'

feature_names = concrete_raw.columns.values.tolist()
feature_names.remove(outcome_name)

print(feature_names)
print(outcome_name)

In [ ]:
# the duplicated method will give a series of true or false if the row is duplicated anywhere
# we can sum this boolean matrix (true = 1, false = 0) to let us know how many rows are duplicates
(
    concrete_raw.loc[:, feature_names]
        .duplicated()
        .sum()
)

In [ ]:
# lets take a strategy to resolve this:
# where we see multiple measurements for the same mixture,
# we will average the observations

concrete = (
   concrete_raw
        .groupby(feature_names)
        .agg('mean')
        .reset_index()
)

print("old dimensions: {} rows, {} columns".format(concrete_raw.shape[0], concrete_raw.shape[1]))
print("new dimensions: {} rows, {} columns".format(concrete.shape[0], concrete.shape[1]))

# Part 2: Exploring your data

Before we even think about modelling, we need to understand what we are dealing with. As we have limited time, we will context some exploration using visualisations.

We will use a library called matplotlib, which is another open sourced project. It is a general purpose plotting library, inspired by matlab.

For a regression task, we want to be able to predict out outcome (Compressive Strength) based on our inputs. Lets try and see if there are any obvious relations between the features and the outcome.

In [ ]:
# load in matplotlib and numpy
import matplotlib.pyplot as plt
import numpy as np

# cell 'magic' which allows us to render our plots in the notebook
%matplotlib inline

## Some basic plotting

In [ ]:
# define two numpy arrays for our x and y data
x = np.arange(10)
y = x * 3

print("x, y pairs are: {}".format(list(zip(x, y))))

In [ ]:
plt.plot(x, y);

In [ ]:
plt.scatter(x, y);

In [ ]:
plt.scatter(x, y, color = "red", label = "i am a label")
plt.xlabel("hat")
plt.ylabel("cat")
plt.legend()
plt.title("I am a title");

Again, matplotlib contains a huge functionality, and the [API documentation](https://matplotlib.org/) is the best place to get started. Also consider these [SciPy lecture notes](https://www.scipy-lectures.org/).

We will use matplotlib to explore our data. Matplotlib works natively with numpy arrays of data, but has in built flexibility to work with pandas as well.

In [ ]:
# lets visualise cement vs compressive strength
# alpha controls the transparency
plt.scatter(x = concrete.loc[:, "Cement"], y = concrete.loc[:, "CompressiveStrength"], alpha = 0.6)
plt.xlabel("Cement")
plt.ylabel("Compressive Strength");

What can we say about the relationship between the amount of cement in the mixture and compressive strength?


## Exercise: Explore the relationships between the features and the outcome

Discuss with your neighbor which features you think may be important to predict compressive strength based on the mixture.

In [ ]:
# Your code here





## Questions

- What can we say about the relationship between the features and the outcome? Do we see any non-linear relationships?

- Do all features look useful

- Do some mixtures lack ingredients?

You may wonder how you can model the compressive strength of the mixture based on the ingredients using such messy looking data. In the next section, we will use a powerful machine learning algorithm that can combine the features to accurately predict the outcome.


# Part 3: Building regression models

In this section, we will use our data to build a regression model to predict the compressive strength of concrete based on the mixture.

We will be using the Scikit-Learn library, the de-facto python library for building machine learning models. It has a clean, consistent API for machine learning algorithms, and various capabilities for data preparation. More advanced users may build machine learning 'pipelines' which pass data through various transformation steps to process it finally making predictions with a model.

Building machine learning algorithms effectively can require a lot of experience; as well as model performance there are many considerations that need to be taken into account, such as how your model will be used once it is built.

If you are interested in building experience in this area, there are a number of sources of data sets, such as UC Irvine ML repository, various high-quality online courses (Coursera, EdX, Udemy etc), and some good blogs.


In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from IPython.display import Image
from IPython.core.display import HTML 

We are going to start by using a model called a random forest.

This model builds a number of decision trees, where each split aims to separate the data into two groups which minimise the sum of squares error:

$$ SSE = \sum_{i \in S_1} (y_i - \bar{y}_1)^2 + \sum_{i \in S_2} (y_i - \bar{y}_2)^2 $$

where $\bar{y}_i$ is the average on the left and the right of the split, and $y_i$ are the observed values of the outcome. 

This is done by searching over every value of every feature in the group to decide which split gives the greatest reduction in error. The fitting is performed recursively until a stopping criteria is reached.

Once the trees are grown, predictions can be made. The new observation instance is scored on every tree in the forest- values are fed through the tree, and every node serves as an if/else statement to decide if the observation should go left or right. When it hits a terminal node (a 'leaf'), the average value of the training data in that node is taken as the tree's prediction. This is then averaged over the results of all the trees in the forest to produce a final prediction.

The trees are decorrelated from one another by training them on randomly sampled (bootstrapped) subsets of the training data, and at each split the set of candidate training features are also randomly selected (m are randomly chosen from n features).

This model is very powerful, and can produce very high quality predictions.

## Preparing our data with a train/test split

One major danger when building models is to overfit to the training data. This means you have trained your model so specifically to the training data, that it will be describing the noise in the training data as well as any general behaviour. A model which is overfit will perform very well on the training data, but will generalise poorly.

To remedy this, we will split our data into an 75:25 split. We will build our model using 75% of the data, and evaluate it on the hold-out 25% of the data. This will give us an indication of the model's true performance.

### Now we have a problem...

We want to model mixture proportions, and we currently have the ingredients by weight. We also have age in days, which is not a mixture ingredient.

Let's normalise our data, so we have ingredients by proportion instead. Pandas comes to the rescue!

In [ ]:
mixture_ingredients = ["Cement", "BlastFurnaceSlag", "FlyAsh",
                       "Water", "Superplasticizer", "CoarseAggregate",
                       "FineAggregate"]


# we divide every ingredient weight by the sum of each mixture
normalised_ingredients = (
    concrete
        .loc[:, mixture_ingredients]
        .div(concrete.sum(axis=1), axis = 0)
)


# join normalised mixture proportion to age and compressive strength
concrete_normalised = pd.merge(
    concrete.loc[:, ["Age", "CompressiveStrength"]],
    normalised_ingredients,
    left_index = True,
    right_index = True
)

In [ ]:
# you will not get the same results if you change the seed here.
# for reproducibility and consistency, you should always seed any function
# that uses a random process
train, test = train_test_split(concrete_normalised, train_size = 0.75, test_size = 0.25, random_state = 3456)

print("train size: {} observations \ntest size: {} observations".format(train.shape[0], test.shape[0]))

## Model fitting

The Scikit-Learn library has a clean, consistent API- check out the docs [here](scikit-learn.org)

It contains an implementation for a random forest regression model, which we can easily fit  using the default parameters.

Most machine learning models have hyperparameters, that is, free parameters than need tuning to find the best possible model. We can start by fitting the default: a forest of 10 trees, which some may consider to be more of a shrubbery than a forest.

In [ ]:
#only set random state, to ensure reproducible results
rf_model = RandomForestRegressor(random_state = 1234)

# whar are the default options?
rf_model

In [ ]:
# lets fit our data to the default model
feature_names = concrete.columns.values.tolist()
outcome_name = "CompressiveStrength"
feature_names.remove(outcome_name)

rf_model.fit(train.loc[:, feature_names], train.loc[:, outcome_name])

### How well do we perform on our training data?

Now our model is fit to the trainig data, we can investigate its performance by plotting the predictions of compressive strength vs the actual values. We can also investigate the distribution of the errors.

Also, we can calculate the root mean square error, which will give us the average absolute error of our predictions in MPa.

In [ ]:
predictions_train = rf_model.predict(train.loc[:, feature_names])
error_train = predictions_train - train.loc[:, outcome_name].values

f, axarr = plt.subplots(1, 2, figsize = (12, 6), dpi=80)

for i in range(2):
    axarr[i].spines['right'].set_visible(False)
    axarr[i].spines['top'].set_visible(False)

    
axarr[0].scatter(predictions_train, train.loc[:, outcome_name].values, alpha = 0.6)
axarr[0].set_xlabel('prediction train')
axarr[0].set_ylabel('outcome train')
axarr[0].set_xlim(0, 90)
axarr[0].set_ylim(0, 90)

axarr[1].scatter(train.loc[:, outcome_name].values, error_train, alpha = 0.6)
axarr[1].set_xlabel('compressive strength')
axarr[1].set_ylabel('prediction train - outcome train')

f.subplots_adjust(wspace = 0.3);


In [ ]:
# what is the rmse on the training set?
print("training set rmse: {}".format(mean_squared_error( train.loc[:, outcome_name].values, predictions_train)** 0.5))

It's important to mention we see an effect called heterosketadicity here- our model's performance quality is not consistent for higher compressive strengths. This isnt suprising, as we have fewer observations for compressive strength > 60, however it is not ideal as we want to build the strongest mixture. This is something we would come back to try and remedy if we were performing a serious study. 

For instance, we could redefine the cost function to more harshly penalise errors when the true value of the mixture is > 60.

### Things are too good to be true...

If we reported these results, we are making the most basic mistake! We have likely overfit to our training data to a degree, and we should evaluate the model on our independent test data set to get an idea of how well the model generalises.

In [ ]:
predictions_test = rf_model.predict(test.loc[:, feature_names])
error_test = predictions_test - test.loc[:, outcome_name].values

f, axarr = plt.subplots(1, 2, figsize = (12, 6), dpi=80)

for i in range(2):
    axarr[i].spines['right'].set_visible(False)
    axarr[i].spines['top'].set_visible(False)

    
axarr[0].scatter(predictions_test, test.loc[:, outcome_name].values, alpha = 0.6)
axarr[0].set_xlabel('prediction test')
axarr[0].set_ylabel('outcome test')
axarr[0].set_xlim(0, 90)
axarr[0].set_ylim(0, 90)

axarr[1].scatter(test.loc[:, outcome_name].values, error_test, alpha = 0.6)
axarr[1].set_xlabel('observation number')
axarr[1].set_ylabel('prediction test - outcome test')

f.subplots_adjust(wspace = 0.3);


In [ ]:
print("test set rmse: {}".format(mean_squared_error( test.loc[:, outcome_name].values, predictions_test)** 0.5))

The test set root mean square error is higher- essentially this means on average, our predictions are $\pm$ 5MPa.
Seeing a higher RMSE for the test set isn't unexpected, as the predictions on the training data have overfit to describe the training data too specifically, and will never perform as well on unseen test data.

## Exercise: hyperparameter tuning

There is never any guarentee that the default hyperparameters are the best choice for the model. Your task is to tune the number of trees, depth of trees, and number of features to randomly select in the model.

I have provided a function which will allow you to test altering the hyperparameters, showing the resulting train and test error.

Your aim is to minimise the RMSE on the test data by experimenting values for n_trees, n_features and max_depth.

In [ ]:
# run this cell, butdont touch any of the code!
def train_test(features_train, outcome_train, 
               features_test, outcome_test,
               n_trees = 100,
               n_features = 3,
               max_depth = 6):
    model  = RandomForestRegressor(n_estimators = n_trees,
                                   max_depth = max_depth,
                                   max_features = n_features,
                                   random_state = 1234)
    
    model.fit(features_train, outcome_train)
    pred_train = model.predict(features_train)
    pred_test = model.predict(features_test)
    
    rmse_train = mean_squared_error(outcome_train, pred_train) ** 0.5
    rmse_test = mean_squared_error(outcome_test, pred_test) ** 0.5

    print("rmse train: {}".format(rmse_train))
    print("rmse test: {}".format(rmse_test))
    
    return 

In [ ]:
# investigate how you can alter n_trees, n_featues and max_depth to beat this default model
# note that there are are only 8 features so n_features cannot exceed this
train_test(train.loc[:, feature_names], train.loc[:, outcome_name], # do not touch
           test.loc[:, feature_names], test.loc[:, outcome_name], # do not touch
           n_trees = 10, 
           n_features = 4,
           max_depth = 6)

## Cross validation

Now, this is a naive way to tune the model hyperparameters. Cross validation is often used for this purpose. There are several variations, but k-fold cross validation is most common, often used in conjunction with a grid search.

The idea is to define a number of combinations of the values of the hyperparameters, which is the grid of values we want to search over. For each combinations of hyperparameter, we split the data into k folds, where we train on $k-1$ and test on the left out fold. We then iterate through all the folds, building and evaluating the model on each combination of the training data. 

This allows us to select the 'best' model on training data, and then we can apply it to the test data set for a final validation. The model would then be ready to apply to any new data samples.

For time considerations we will not investigate now, but if anyone is interested, do investigate the GridSearchCV class in the scikit-learn library.

## As well as the RF model, we will also train a GBR model

Think of this as a complex 'black box' model. It can produce very high quality predictions when tuned properly. We dont have time to discuss now, but if you are interested to understand how boosting works, the scikit-learn documentation is a good place to start. 'AdaBoost' is probably the most intuitive boosting scheme. We focussed on the random forest model previously as it is easier to understand intuitively.

The GBR model also makes high quality predictions when tuned well, and can also be much much faster to make predictions once trained than a random forest. This is important for the mixture optimisation we will perform in the next part of this tutorial. 

In [ ]:
gbr = GradientBoostingRegressor(n_estimators = 1000, max_depth = 2, random_state = 1234)
gbr.fit(train.loc[:, feature_names], train.loc[:, outcome_name])

# Part 4: Optimising mixtures

Our aim was to be able to predict mixtures that would lead to optimal compressive strengths to guide experminentation. We now have a model that can predict compressive strength based on mixture proportions, and the final step is to use an optimiser to find the mixture that produces the strongest concrete.

We will use the [Scipy library](https://www.scipy.org/), which contains an optimiser for this purpose. We firstly have to define a cost function which we want to minimise, and then we will run the optmiser from different starting points to find our candidate mixtures. Using different starting points is important, as we have no guarentee to find the global minimum.

In [ ]:
from scipy.optimize import minimize

## We will look at 28 day old samples
 
We saw age was a feature, and is not part of a mixture. This is because concrete takes time to properly cure. As we had many observations for 28 day old samples, we will limit our investigation to these. 

In [ ]:
mixture_ingredients = ['Cement','BlastFurnaceSlag','FlyAsh','Water',
                       'Superplasticizer','CoarseAggregate','FineAggregate','Age']

test_age_28 = test.loc[test.Age == 28, mixture_ingredients]

## Define a function to optimise

The optimiser needs a cost ('objective') function to work with. It will take this function and some starting values, and try to find a minimum.

The cost function takes in n-1 of our mixture proportions, and infers the last. We do this to ensure our mixture proportions sum exactly to 1. Once that process is complete, we append the age (28) to our feature vector. This is then fed into our model, and a prediction is generated. The optimiser will explore mixtures from a given starting point to try and find an optimal value, i.e. the mixture proportions which yield the greatest compressive strength.

As the optimiser is searching for a minimum, we return the negated compressive strength.

In [ ]:
# assume numpy array
# x[0] = Cement
# x[1] = BlastFurnaceSlag
# x[2] = FlyAsh
# x[3] = Water
# x[4] = Superplasticizer
# x[5] = CoarseAggregate

def cost_function(x, model):
    
    # Check to make sure the mixture proportions are in the correct range
    # 'explode' the cost function if we violate
    if(x[0] < 0.0 or x[1] > 1.0): return(10**38)
    if(x[1] < 0.0 or x[1] > 1.0): return(10**38)
    if(x[2] < 0.0 or x[2] > 1.0): return(10**38)
    if(x[3] < 0.0 or x[3] > 1.0): return(10**38)
    if(x[4] < 0.0 or x[4] > 1.0): return(10**38)
    if(x[5] < 0.0 or x[5] > 1.0): return(10**38)
    
    # x[6] = FineAggregate, the proportion is 1 - the rest
    x = np.append(x, (1 - np.sum(x)))
    
    # if unrealistic amount of water
    if(x[3] < 0.05): return(10**38)

    # add age = 28
    x = np.append(x, 28.0)
    
    # return the prediction from the model
    # negate as we want the largest compressive strength, 
    # and the optimiser will minimise
    return -model.predict(x[None, :])[0]

## Mixture optimisation


Our strategy is to use  each observation in test set as a starting point (as we know they are realistic mixtures) and try to optimise.

Our cost function is not smooth, so we need to be careful which method we use. Nealder-Mead should perform well here.

We see not all starting values lead to a solution. We will run an optimisation for each starting value, and return the mixtures which we predict will give the best results.

In [ ]:
res = np.empty((115, 9))
cols = ['Cement','BlastFurnaceSlag','FlyAsh','Water','Superplasticizer','CoarseAggregate']
index_vals = test_age_28.index.values

for i, ix in enumerate(index_vals):
    
    # current starting values
    curr_values = test_age_28.loc[ix, cols].values
    
    # perform the optimization
    opt = minimize(cost_function, curr_values, args=(gbr), method = "Nelder-Mead", options={'maxiter': 5000})
    
    if i % 25 == 0:
        print("===============")
        print("starting sample", i)
        print("compressive strength:",  -opt.fun)
        print("mixture:", opt.x)
        print()
    
    # store our mixtures and prediction of compressive strength
    res[i,:6] = opt.x
    res[i, 6] = (1.0 - opt.x.sum())
    res[i, 7] = 28.0
    res[i, 8] = -opt.fun
      

In [ ]:
# lets pull out our best predictions
(
    pd.DataFrame(res, columns = mixture_ingredients + ['CompressiveStrength'])
        .sort_values(by = 'CompressiveStrength', ascending = False)
).head()

# Summary

We have achieved our objective: we have produced a selection of mixes we can propose for further experimentation.

To expand this work, we could include other consideration, for instance financial cost. The cost function could be enhanced with desirability function. For instance, if cement is very expensive, the fourth strongest mix prediction would be prohibitave.

